# Proyecto

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc, make_scorer

In [32]:
df_x0 = pd.read_parquet('X_t0.parquet')
df_x1 = pd.read_parquet('X_t1.parquet')
df_y0 = pd.read_parquet('y_t0.parquet')

In [3]:
# profile = ProfileReport(
#     df_x0,
#     title="Análisis Exploratorio de Datos Automático",
#     explorative=True
# )

# # Guarda el informe como un archivo HTML
# profile.to_file("EDA_Reporte.html")

In [4]:
rd = 1323

In [ ]:
# Asegurarnos de que la variable objetivo sea un vector
y = df_y0.values.ravel()

# Eliminar la columna de texto
text_column = 'wallet_address'
X = df_x0.drop(columns=[text_column])

# Definir una métrica personalizada para AUC-PR
def custom_auc_pr(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Crear un scorer basado en la métrica personalizada
auc_pr_scorer = make_scorer(custom_auc_pr, response_method = 'predict_proba')

# Definir un modelo base
xgb = XGBClassifier(eval_metric='logloss', random_state=rd, scale_pos_weight = 1)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'reg_alpha': [0, 0.1, 1]  
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X, y)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 1}
Mejor puntuación de validación cruzada (AUC-PR): 0.9473308134154177


In [6]:
from catboost import CatBoostClassifier

# Definir un modelo base
catboost = CatBoostClassifier(
    verbose=0,
    random_seed=rd
)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'iterations': [100, 200],
    'depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'l2_leaf_reg': [1, 3, 5] 
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=catboost,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X, y)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")    

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros: {'depth': 5, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Mejor puntuación de validación cruzada (AUC-PR): 0.9319980463592461


In [7]:
from lightgbm import LGBMClassifier

# Definir un modelo base
lightgbm = LGBMClassifier(
    random_state=rd
)

# Definir un rango reducido de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],  
    'max_depth': [3, 5],       
    'learning_rate': [0.01, 0.1],
    'reg_alpha': [0, 0.1, 1]    
}

# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=lightgbm,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X, y)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
[LightGBM] [Info] Number of positive: 22656, number of negative: 21640
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17153
[LightGBM] [Info] Number of data points in the train set: 44296, number of used features: 76
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511468 -> initscore=0.045881
[LightGBM] [Info] Start training from score 0.045881
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

In [11]:
# Cargar los datos
df_x0 = pd.read_parquet('X_t0.parquet')
df_y0 = pd.read_parquet('y_t0.parquet')

# Asegurarnos de que la variable objetivo sea un vector
y = df_y0.values.ravel()

# Eliminar la columna de texto
text_column = 'wallet_address'
X = df_x0.drop(columns=[text_column])

# Definir una métrica personalizada para AUC-PR
def custom_auc_pr(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Crear un scorer basado en la métrica personalizada
auc_pr_scorer = make_scorer(custom_auc_pr, response_method = 'predict_proba')

# Definir un modelo base
xgb = XGBClassifier(eval_metric='logloss', random_state=rd, scale_pos_weight = 1)

# Definir una grilla ampliada de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 1, 10],
    'reg_lambda': [1, 5, 10],
    'subsample': [0.5, 1]
}


# Configurar la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rd)

# Definir el GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=auc_pr_scorer, 
    cv=cv,
    verbose=10,
    n_jobs=-1
)

# Ejecutar el grid search
print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X, y)

# Imprimir los mejores parámetros y el mejor puntaje
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor puntuación de validación cruzada (AUC-PR): {grid_search.best_score_}")

Iniciando búsqueda de hiperparámetros...
Fitting 5 folds for each of 1536 candidates, totalling 7680 fits
Mejores parámetros: {'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 500, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 1}
Mejor puntuación de validación cruzada (AUC-PR): 0.9769652787074126


In [54]:
from zipfile import ZipFile
import os

# Obtener el mejor modelo de grid search
best_model = grid_search.best_estimator_
X1 = df_x1.drop(columns=[text_column])

def generateFiles(predict_data, clf_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    ---------------
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf

    Ouput
    ---------------
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict_proba(predict_data)[:, 1]
    with open('./predictions.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)
    
    with ZipFile('predictions.zip', 'w') as zipObj:
        zipObj.write('predictions.txt')
    os.remove('predictions.txt')

generateFiles(X1, best_model)

In [55]:
import joblib

# Guardar el mejor modelo
joblib.dump(best_model, 'best_model.pkl')

print("El mejor modelo ha sido guardado en 'best_model.pkl'")

El mejor modelo ha sido guardado en 'best_model.pkl'
